# Le jeu de dobble

## présentation

Le dobble est un jeu de cartes:

* chaque carte possède huit symboles,
* quelque soit une paire de cartes, elles ont en commun exactement un symbole

![dobble](dobble.png)

## données brutes

On vous donne la liste des cartes, dans un ordre totalement aléatoire:

In [ ]:
# il y en a davantage que ça, mais pour vous donner une idée:
# une carte par ligne
!head -8 cards.raw

## construction du paquet de cartes

In [ ]:
# une carte est un ensemble de symboles
# et un symbole est représenté par une simple chaine
class Card(set):
    """
    le modèle pour chaque carte du jeu
    """
    
    # on leur donne un numéro arbitraire
    # dans l'ordre du paquet 
    counter = 1
    
    def __init__(self, *args, **kwds):
        set.__init__(self, *args, **kwds)
        self.counter = Card.counter 
        Card.counter += 1
        
    def __repr__(self):
        return f"[{self.counter:2d}] " + set.__repr__(self)
    
    def __hash__(self):
        return self.counter

In [ ]:
def read_cards():
    with open('cards.raw') as f:
        return [Card(line.split()) for line in f]

Attention à ne pas utiliser juste `cards` parce que c'est un nom de variable qu'on va massivement utiliser

In [ ]:
all_cards = read_cards()
print(f"we have {len(all_cards)} cards")

### combien de symboles

In [ ]:
symbols = set()
for card in all_cards:
    symbols = symbols | card
print(f"we have {len(symbols)} symbols")

In [ ]:
# la liste des symboles, un peu mise en forme
columns = 7
for i, symbol in enumerate(symbols):
    print(f"{symbol:16s}", end="")
    if (i+1) % columns == 0:
        print()

## vérifications

#### toutes les cartes ont 8 symboles

In [ ]:
for card in all_cards:
    if len(card) != 8:
        print(f"OOPS {card} -> {len(card)}")

#### exactement un point commun entre 2 cartes quelconques

In [ ]:
# un table de hash : card1, card2 -> symbole
common_symbol = {}

# on range les conflits par cardinal de l'intersection (0 ou 2)
for c1 in all_cards:
    for c2 in all_cards:
        # comme on est sûr que les deux boucles se font
        # dans le même ordre, on peut mettre break 
        # si on fait continue, on a deux fois trop de couples 
        if c1 is c2:
            continue
        # combien de cartes en commun
        common = (c1 & c2)
        if len(common) != 1:
            print(f"--- between {c1} and {c2}: {common} common items:\n")
            print(common)
        else:
            common_symbol[c1, c2] = common.pop()

### symboles les plus utilisés

In [ ]:
from collections import defaultdict

`symbol_to_cards` : un hash (dictionnaire) qui associe à un symbole l'ensemble des cartes où il apparaît

In [ ]:
symbol_to_cards = defaultdict(set)

for card in all_cards:
    for symbol in card:
        symbol_to_cards[symbol].add(card)

On le trie par fréquence d'apparition :

In [ ]:
# symbol_cards_list est une liste de tuples de la forme
# symbole, [carte1, carte2, ...]
symbol_cards_list = list(symbol_to_cards.items())
symbol_cards_list[:2]

In [ ]:
# on le trie sur la taille de la partie droite du tuple 
symbol_cards_list.sort(key=lambda item: len(item[1]))
symbol_cards_list[:2]

In [ ]:
# de nouveau on essaie d'afficher tout ça sur une page
# les cartes qui apparaissent le moins sont en premier

columns = 5

for i, (symbol, scards) in enumerate(symbol_cards_list):
    print(f"{symbol:>15s} [{len(scards)}] ", end="")
    if (i+1) % columns == 0:
        print()    

## les cartes en fonction des symboles

Pour montrer la même information mais avec le détail des cartes.  
Par exemple, on sait que `bonhommeneige` apparait sur 6 cartes mais maintenant on veut voir lesquelles:

In [ ]:
cards = list(symbol_cards_list[0][1])[0].counter

In [ ]:
# en vrac
if True:
    for symbol, cards in symbol_cards_list:
        print(f"{symbol:15s} ", end="")
        print(" - ".join(f"{card.counter:02d}" for card in sorted(cards, key=lambda card: card.counter)))

## nombre de fois qu'un symbole est un point commun

In [ ]:
occurrences = defaultdict(int)

for c1 in all_cards:
    for c2 in all_cards:
        if c1 is c2:
            # si on mettait continue ici on n'aurait le bon nombre mais double
            break
        common = common_symbol[c1, c2]
        occurrences[common] += 1

In [ ]:
# de nouveau on essaie d'afficher tout ça sur une page
# les cartes qui apparaissent le moins sont en premier

less_often_first = sorted(occurrences.items(), key=lambda couple: couple[1])

columns = 5

for i, (symbol, occurrences) in enumerate(less_often_first):
    print(f"{symbol:>15s} [{occurrences}] ", end="")
    if (i+1) % columns == 0:
        print()    

## une petite vérification

In [ ]:
# en tout on a un nombre de paires de cartes
number_cards = len(all_cards)
total_pairs = number_cards * (number_cards-1) // 2

total_pairs

In [ ]:
# qui doit correspondre avec la somme des occurrences de points communs 
# qu'on vient de calculer
sum(couple[1] for couple in less_often_first)

## une remarque

C'est troublant tout de même que tous ces nombres d'occurrences font partie de la même suite:

In [ ]:
# (1, 3, 6, 10,) 15, 21, 28
for n in range(1, 8):
    print(n*(n+1)//2)